In [129]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

In [172]:
train_features_data=pd.read_csv('train_features.csv')
train_labels_data=pd.read_csv('train_labels.csv')
test_features_data=pd.read_csv('test_features.csv')
# add labels with matching pid to features
labels_copied = pd.DataFrame()
labels_copied = train_labels_data.loc[train_labels_data.index.repeat(12)]
labels_copied = labels_copied.drop(columns=['pid'])
labels = train_labels_data.drop(columns=['pid'])
LABELS1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total',
         'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2',
         'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
LABELS2 = ['LABEL_Sepsis']
LABELS3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

## Subtask 1

In [173]:
X = train_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(train_features_data.mean())
y = labels_copied[LABELS1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

TEST_X = test_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(test_features_data.mean())

### RandomForest model and Roc_Auc_Score using every hour per patient as sample for training

In [149]:
clf=RandomForestClassifier(n_estimators=100, class_weight=None, n_jobs=-1 )
clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=np.empty(10)
list_proba=list()
for i,label in enumerate(LABELS1):
    err[i]=roc_auc_score(y_test[label], y_pred_proba[i][:,1])
    list_proba.append(y_pred_proba[i][:,1])
display(err)
display('Average score: ', np.mean(err))

array([0.8371992 , 0.73907915, 0.70539894, 0.70741244, 0.7055589 ,
       0.75526489, 0.80465126, 0.78276616, 0.78734367, 0.90152352])

'Average score: '

0.7726198144723001

### Predictions for Test_features

In [118]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [119]:
TEST_list_proba=list()
for j in range(10):
    TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[j][:,1])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_proba[j][:,1]), int(len(TEST_y_pred_proba[j][:,1])/12)):
        TEST_y_pred_proba_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_proba.append(TEST_y_pred_proba_reduced)
proba_subtask1=TEST_list_proba

## Subtask 2

In [170]:
y = labels_copied[LABELS2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

### RandomForest model and Roc_Auc_Score using every hour per patient as sample

In [171]:
clf=RandomForestClassifier(n_estimators=100, class_weight=None, n_jobs=-1)

clf.fit(X_train, y_train)
y_pred_proba=clf.predict_proba(X_test)
err=roc_auc_score(y_test, y_pred_proba[:,1])
display(err)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

### Prediction for Test features

In [122]:
TEST_y_pred_proba=clf.predict_proba(TEST_X)

### Reduce values by 12 for TEST_y_pred_proba

In [123]:
TEST_y_pred_proba_reduced=np.empty(int(len(TEST_y_pred_proba[:,1])/12))
counter=0
for splits in np.split(np.array(TEST_y_pred_proba[:,1]), int(len(TEST_y_pred_proba[:,1])/12)):
    TEST_y_pred_proba_reduced[counter]=splits.mean() 
    counter=counter+1
proba_subtask2=TEST_y_pred_proba_reduced

## Subtask 3

In [124]:
y = labels_copied[LABELS3]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=7)

### Implement MultiTaskLassoCV model

In [125]:
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score


cv = RepeatedKFold(n_splits=10, n_repeats=3)

model = RidgeCV(alphas=(0.1, 1, 10), cv=cv)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.1767142584906002
71.33924591744208


### Predictions for Test features

In [126]:
TEST_y_pred_values=model.predict(TEST_X)
display(TEST_y_pred_values)

array([[17.8862018 , 85.20264956, 97.14079226, 80.91155894],
       [14.96170099, 81.91628556, 97.78323708, 85.89329861],
       [17.74653736, 83.01390728, 97.34778382, 86.12383761],
       ...,
       [18.62797997, 82.27013909, 97.03111402, 86.63179763],
       [18.79326927, 80.66693961, 97.02444702, 86.48322337],
       [18.26002063, 82.63656682, 97.0866548 , 86.1186615 ]])

### Reduce values by 12 for TEST_y_pred_values

In [127]:
TEST_list_values=list()
for j in range(4):
    TEST_y_pred_values_reduced=np.empty(int(len(TEST_y_pred_values[:,j])/12))
    counter=0
    for splits in np.split(np.array(TEST_y_pred_values[:,j]), int(len(TEST_y_pred_values[:,j])/12)):
        TEST_y_pred_values_reduced[counter]=splits.mean() 
        counter=counter+1
    TEST_list_values.append(TEST_y_pred_values_reduced)
proba_subtask3=TEST_list_values

### Write values into sample dataframe

In [128]:
filename = 'sample.zip'
df_submission = pd.read_csv(filename)
for i,label in enumerate(LABELS1):
    # round classification labels
    df_submission[label]=proba_subtask1[i]
df_submission[LABELS2[0]]=proba_subtask2
for i,label in enumerate(LABELS3):
    # round classification labels
    df_submission[label]=proba_subtask3[i]
df_submission.to_csv('submission.csv',index=False)
df_submission.to_csv('prediction.zip', index=False, float_format='%.3f', compression='zip')